<a href="https://colab.research.google.com/github/Hamada-Gado/IEEE-GUC-Deep-Learning-Tasks/blob/main/IEEE_Session_3_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>In this notebook, we're going to use Pytorch to train a few classifiers for the Fashion-MNIST dataset.</h3>

The classifiers we'll implement are: 

1. Linear Classifier (Logistic Regression as a Neural Network)

2. Deep Neural Network (Adding non-linearity via an activation function)

3. A DNN with a different parameter initialization (Xavier initialization)

4. A DNN with l2 regularization (weight decay)

5. A DNN with dropout




<p>While these are a total of 5 classifiers, the differences in implementation are a few lines at most, sometimes even a single argument!

Note: It's generally unadvisable to flatten images into a single dimension and treat them as a 1D vector without exploiting the spatial structrue in the image in a more satisfying way. 

We do this here for practice and for educational purposes, but a better way is to use Convolutional Neural Networks, which we discuss later.</p>

Also note that all networks will take from 2 to 10 minutes to train. This is because we're training on CPU which generally uses much more time for neural network training than a GPU.

We'll start by importing PyTorch and some additional packages we'll need:

In [ ]:
import torch
import torchvision
import numpy as np
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# set default device based on CUDA's availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


#Dataset Class

Next we'll define our dataset class. The dataset we're using is Fashion-MNIST, which is included in torchvision's datasets. We just have to call the appropiate function to downlaod it.

We also make use of torchvision's transforms module to prepare the images by resizing them if needed and converting the to Pytorch tensors.

We use torch's built-in DataLoader to get dataloaders for the training and validation sets and add some helper methods for visualizing the dataset.

In [ ]:
class FashionMNIST(Dataset):
    """The Fashion-MNIST dataset."""
    def __init__(self, batch_size=64, resize=(28, 28), root=".", num_workers=2):
        super().__init__()

        self.batch_size = batch_size
        self.resize = resize
        self.root = root
        self.num_workers = num_workers

        trans = transforms.Compose([transforms.Resize(resize),
                                    transforms.ToTensor()])
        self.train = torchvision.datasets.FashionMNIST(
            root=self.root, train=True, transform=trans, download=True)
        self.val = torchvision.datasets.FashionMNIST(
            root=self.root, train=False, transform=trans, download=True)


    def text_labels(self, indices):
      """Return text labels."""
      labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
      return [labels[int(i)] for i in indices]


    def get_dataloader(self, train):
        data = self.train if train else self.val
        return torch.utils.data.DataLoader(data, self.batch_size, shuffle=train,
                                          num_workers=self.num_workers)

    def train_dataloader(self):
        return self.get_dataloader(train=True)

    def val_dataloader(self):
        return self.get_dataloader(train=False)

    def show_images(self, imgs, num_rows, num_cols, titles=None, scale=1.5):
        figsize = (num_cols * scale, num_rows * scale)
        _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
        axes = axes.flatten()
        for i, (ax, img) in enumerate(zip(axes, imgs)):
            try:
                img = np.array(img)
            except:
                pass
            ax.imshow(img)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            if titles:
                ax.set_title(titles[i])
        return axes

    def visualize(self, batch, nrows=1, ncols=8, labels=[]):
        X, y = batch
        if not labels:
            labels = self.text_labels(y)
        self.show_images(X.squeeze(1), nrows, ncols, titles=labels)

Now we can instantiate the dataset class and get some batches of data and visualize them.

In [ ]:
data = FashionMNIST()
#Total number of training, validation examples
len(data.train), len(data.val)

In [ ]:
#Shape of a single image
data.train[0][0].shape

In [ ]:
X, y = next(iter(data.train_dataloader()))
#Shape of a single batch with batch size = 64
print(X.shape, X.dtype, y.shape, y.dtype)

In [ ]:
batch = next(iter(data.val_dataloader()))
data.visualize(batch)

#First classifier: Linear Network

We'll start by defining our first classifier: a simple logistic regression model implemented as a neural network with a single linear layer.


Note that in PyTorch we don't explicitly include the nn.Softmax in the net architecture. This is because PyTorch's built-in [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) expects the input to be unnormalized logits (i.e normal linear layer output without applying softmax)

We don't need to apply Softmax during validation unless we require class probabilities, otherwise we can just get the maximum probability class with argmax in which case we don't need to calculate the Softmax before (as Softmax preserves ordering so the max argument before Softmax is the same after)



**Q1: Implement a linear neural network for classification**

In [ ]:
class SoftmaxRegression(nn.Module):  
    def __init__(self, num_outputs, lr):
        super().__init__()

        self.num_outputs=num_outputs
        self.lr = lr


        self.net = nn.Sequential(
          #TODO: Add a nn.Flatten() layer

          #TODO: Add a nn.Linear layer after it with input 28*28
          #      and output=num_outputs

        )

    def forward(self, X):
        return self.net(X)

    def loss(self, Y_hat, Y, averaged=True):
      Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
      Y = Y.reshape((-1,))
      return F.cross_entropy(
          Y_hat, Y, reduction='mean' if averaged else 'none')

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)



Now what remains is to define the trainer class. Note that you can directly write the training loop without wrapping it in a class, but doing it this way is cleaner and more modular. We will use this same trainer class to train all classifiers in this notebook without having to rewrite it.

**Q2: Complete the Trainer class**

In [ ]:
class Trainer:
    def __init__(self, max_epochs, num_gpus=0):
        assert num_gpus == 0, 'No GPU support yet'

        self.max_epochs = max_epochs

        #We use those lists to plot the loss and accuracy
        self.train_losses = []
        self.val_losses = []

        self.train_acc = []
        self.val_acc = []


    def prepare_data(self, data):

      self.train_dataloader = data.train_dataloader()
      self.val_dataloader = data.val_dataloader()

      self.num_train_batches = len(self.train_dataloader)
      self.num_val_batches = (len(self.val_dataloader)
                              if self.val_dataloader is not None else 0)
        

    def fit(self, model, data):

        self.prepare_data(data)
        self.model = model
        self.optim = model.configure_optimizers()

        self.epoch = 0
        self.batch_size = data.batch_size

        #Train for max_epochs epochs
        for self.epoch in range(self.max_epochs):
            self.fit_epoch()

        self.plot_loss()
        self.plot_accuracy()
        self.visualize_wrong()

    def fit_epoch(self):
        #TODO: Call the train() method of self.model
        

        running_loss = 0
        running_acc = 0

        for batch in self.train_dataloader:

            X, y = batch

            #TODO: Calculate y_hat by applying self.model on X
            y_hat = 


            #TODO: Calculate loss by calling self.model.loss
            loss = 


            #Accumulate loss each batch in running_loss
            running_loss += loss.item()

            #TODO: Get predictions by calling torch.argmax on y_hat with axis=1
            predicted = 

            #Calculate average accuracy for a single batch
            running_acc += (predicted == y).sum().item() / self.batch_size

            #TODO: Zero out self.optim's gradient with the zero_grad() method
            


            #TODO: Call loss.backward 


            #TODO: Call self.optim's step method


        #Divide loss and accuracy by number of training batches per epoch
        running_loss /= self.num_train_batches
        running_acc /= self.num_train_batches

        self.train_losses.append(running_loss)
        self.train_acc.append(running_acc)

        #TODO: Call the eval() method of self.model

        

        val_running_loss = 0
        val_running_acc = 0

        for batch in self.val_dataloader:
            with torch.no_grad():

                X, y = batch

                #TODO: Calculate y_hat by applying self.model on X
                y_hat = 
                #TODO: Calculate loss by calling self.model.loss
                loss = 
                #TODO: Get predictions by calling torch.argmax on y_hat with axis=1    
                predicted = 

                val_running_acc += (predicted == y).sum().item() / self.batch_size

                val_running_loss += loss.item()
        
        val_running_loss /= self.num_val_batches
        val_running_acc /= self.num_val_batches

        self.val_losses.append(val_running_loss)
        self.val_acc.append(val_running_acc)

        print(f"Epoch: {self.epoch}, train loss: {running_loss}, val loss: {val_running_loss} \
                train acc: {running_acc} val acc: {val_running_acc}")


    def plot_loss(self):
      """Plot loss for each epoch"""
      plt.plot(self.train_losses, label='train loss')
      plt.plot(self.val_losses, label='val loss', )
      plt.legend()
      plt.show()

    def plot_accuracy(self):
      """Plot accuracy for each epoch"""
      plt.plot(self.train_acc, label='train accuracy')
      plt.plot(self.val_acc, label='val accuracy')
      plt.legend()
      plt.show()

    def visualize_wrong(self):
      """Visualize wrong examples for a single batch
          The first line is the true class and 
          the second is the predicted class"""
      X, y = next(iter(self.val_dataloader))
      preds = self.model(X).argmax(axis=1)

      wrong = preds.type(y.dtype) != y
      X, y, preds = X[wrong], y[wrong], preds[wrong]
      labels = [a+'\n'+b for a, b in zip(
          data.text_labels(y), data.text_labels(preds))]
      data.visualize([X, y], labels=labels)

    

Now for the training part. We instantiate the dataset, model, and trainer classes, and call the fit method of the trainer class on the model and the data.

**Q3: Initialize the dataset class, model, trainer, and finall call the fit method of the trainer**

In [ ]:
data = #TODO: Instantiate the dataset class defined above with batch_size=256
model = #TODO: Instantiate the model class with num_outputs=10 and lr=0.1
trainer = #TODO: Instantiate the trainer class with max_epochs = 10


#TODO: Call the fit method of the trainer class


#Second Classifier: Multilayer Perceptron (MLP) 

**Q4: Implement MLP by adding a hidden layer and a ReLU activation to the Linear Model**

In [ ]:
class MLP(nn.Module):
    def __init__(self, num_outputs, num_hiddens, lr):
        super().__init__()

        self.num_num_outputs = num_outputs
        self.num_hiddens = num_hiddens
        self.lr = lr

        self.net = nn.Sequential(
        #TODO: Add a nn.Flatten()
        #TODO: Add a nn.Linear with input=28*28 and output=num_hiddens

        #TODO: Add a nn.ReLU after the Linear Layer you define
        #TODO: Add a nn.Linear with input=num_hiddens and output=num_outputs
                                 
                                 )

    def forward(self, X):
        return self.net(X)

    def loss(self, Y_hat, Y, averaged=True):
      Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
      Y = Y.reshape((-1,))
      return F.cross_entropy(
          Y_hat, Y, reduction='mean' if averaged else 'none')

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)


data = FashionMNIST(batch_size=256)
model = MLP(num_outputs=10, num_hiddens=256, lr=0.1)
trainer = Trainer(max_epochs=10)
trainer.fit(model, data)

#Third Classifier: MLP with Xavier initialization

**Q5: Add Xavier initialization to the MLP**

To apply Xavier initialization we define the net as usual and then call the apply method of the net which applies its argument to all layers in the model. 

We check if the layer in question is a Linear layer and if so we call `torch.nn.init.xavier_uniform_` on layer.weight to update it in place.

In [ ]:
class MLPXavier(nn.Module):
    def __init__(self, num_outputs, num_hiddens, lr):
      super().__init__()

      self.num_num_outputs = num_outputs
      self.num_hiddens = num_hiddens
      self.lr = lr

      #TODO: Use the same net you used in Q4
      self.net = nn.Sequential(
          

      )
      
      #TODO: Call self.net.apply and pass the init_weights method as an argument
      

    def init_weights(self, layer):
        if isinstance(layer, nn.Linear):
            #TODO: call torch.nn.init.xavier_uniform_ on layer.weight

            layer.bias.data.fill_(0.01)

    def forward(self, X):
        return self.net(X)

    def loss(self, Y_hat, Y, averaged=True):
      Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
      Y = Y.reshape((-1,))
      return F.cross_entropy(
          Y_hat, Y, reduction='mean' if averaged else 'none')

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)



data = FashionMNIST(batch_size=256)
model = MLPXavier(num_outputs=10, num_hiddens=256, lr=0.1)
trainer = Trainer(max_epochs=10)
trainer.fit(model, data)

#Fourth Classifier: MLP with weight decay (l2 regularization)

**Q6: Add weight decay to the MLP model**

Weight decay helps regularize the model by constraining the values of the weights. We can apply weight decay in Pytorch throught the weight_decay argument of the SGD optimizer.

In [ ]:
class MLPWeightDecay(nn.Module):
    #TODO: Add a wd parameter to the init method with a default value of 0.001
    def __init__(self, num_outputs, num_hiddens, lr):
        super().__init__()

        self.num_num_outputs = num_outputs
        self.num_hiddens = num_hiddens
        self.lr = lr
        #TODO: Add the wd parameter as a class attribute to use in the optimizer

        #TODO: Use the same net you used in Q4 and Q5
        self.net = nn.Sequential(

                                  )

    def forward(self, X):
        return self.net(X)

    def loss(self, Y_hat, Y, averaged=True):
      Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
      Y = Y.reshape((-1,))
      return F.cross_entropy(
          Y_hat, Y, reduction='mean' if averaged else 'none')

    def configure_optimizers(self):
        #TODO: Add a weight_decay argument to the SGD call with value self.wd
        return torch.optim.SGD(self.parameters(), lr=self.lr)


data = FashionMNIST(batch_size=256)
model = MLPWeightDecay(num_outputs=10, num_hiddens=256, lr=0.1)
trainer = Trainer(max_epochs=10)
trainer.fit(model, data)


#Fifth Classifier: MLP with dropout

**Q7: Add dropout to the MLP model**

We use dropout to turn off some neurons with a probability p to regularize the model and reduce overfitting.

In [ ]:
class MLPDropout(nn.Module):
    #TODO: Add a dropout parameter with a default value of 0.5
    def __init__(self, num_outputs, num_hiddens, lr):
        super().__init__()

        self.num_num_outputs = num_outputs
        self.num_hiddens = num_hiddens
        self.lr = lr
        
        #TODO: Use the same net in Q4, Q5, and Q6

        #TODO: Add a nn.Dropout(p) after the ReLU activation
        #      where p is the dropout parameter
        self.net = nn.Sequential(
            
                                 )

    def forward(self, X):
        return self.net(X)

    def loss(self, Y_hat, Y, averaged=True):
      Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
      Y = Y.reshape((-1,))
      return F.cross_entropy(
          Y_hat, Y, reduction='mean' if averaged else 'none')

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)


data = FashionMNIST(batch_size=256)
model = MLPDropout(num_outputs=10, num_hiddens=256, lr=0.1)
trainer = Trainer(max_epochs=10)
trainer.fit(model, data)


<h1>That's it!</h1>
This is optional, but feel free to change the number of hidden units in each hidden layer, change various hyperparameters like dropout probability, learning rate, batch size, and number of epochs. 

You can also experiment with adding more hidden layers and seeing how the loss and accuracy change. Does increasing regularization help when increasing the number of hidden layers?

Attribution: Code in this notebook was based on code from [Dive Into Deep Learning](https://d2l.ai/)